In [1]:
import tensorflow as tf 
from tensorflow import keras as keras
from tensorflow.python.keras import backend as K
import os
import numpy as np
import datetime
import importlib


import Clone_Model
import rsc

In [5]:
#tf.compat.v1.disable_eager_execution()

In [2]:
checkpoint_path = "./cloning_eager_dis/checkpoints/cloning/default_checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

log_dir_clone = "./cloning_eager_dis/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir_clone, histogram_freq=1)

In [14]:
#      CLONE MODEL EAGER DISABLED
###########################################
importlib.reload(rsc)
importlib.reload(Clone_Model)

#K.clear_session() 

#tf.compat.v1.disable_eager_execution()

batch_size = None #doesnt work/conflict tensorboard_callback
train, test = rsc.get_titanic_dataset()



model = rsc.get_model(None)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    run_eagerly = tf.executing_eagerly()
    )


tensorboard_callback.set_model(model)

history = model.fit(train,
                    batch_size=batch_size,
                    epochs=1, 
                    steps_per_epoch=3, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[])


clone = Clone_Model.satify_model(model)

clone.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              run_eagerly = tf.executing_eagerly())

sat_cb = Clone_Model.sat_results()


history = clone.fit(train,
                    batch_size=batch_size,
                    epochs=1,
                    steps_per_epoch=3,
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[sat_cb])
                

Train on 3 steps, validate on 1 steps
3/3 [==============================] - 0s 23ms/step - loss: 3.1866 - val_loss: 1.7910
RESET Weights!
RESET Weights!
RESET Weights!
RESET Weights!
RESET Weights!


ValueError: Error when checking input: expected model_input to have 2 dimensions, but got array with shape (1, 28, 28)

In [3]:
#      CLASSIFY MNIST EAGER DISABLED
###########################################
importlib.reload(rsc)
importlib.reload(Clone_Model)

K.clear_session() 
print(tf.executing_eagerly())

#problem source
#tf.compat.v1.disable_eager_execution()

batch_size = 20 

train, test = rsc.get_mnist()

model = rsc.get_model_unitlist(hidden_layers_spec=[128])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
    run_eagerly = False
    )


tensorboard_callback.set_model(model)

history = model.fit(train[0],
                    train[1],
                    batch_size=batch_size,
                    epochs=5, 
                    steps_per_epoch=None, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[])


True
Epoch 1/5
3000/3000 [==============================] - 7s 2ms/step - loss: 2.0697 - accuracy: 0.8417 - val_loss: 0.2230 - val_accuracy: 0.9000
Epoch 2/5
3000/3000 [==============================] - 7s 2ms/step - loss: 0.3857 - accuracy: 0.9061 - val_loss: 0.1445 - val_accuracy: 0.9500
Epoch 3/5
3000/3000 [==============================] - 11s 4ms/step - loss: 0.3005 - accuracy: 0.9265 - val_loss: 0.2273 - val_accuracy: 0.9000
Epoch 4/5
3000/3000 [==============================] - 7s 2ms/step - loss: 0.2692 - accuracy: 0.9336 - val_loss: 0.0478 - val_accuracy: 1.0000
Epoch 5/5
3000/3000 [==============================] - 8s 3ms/step - loss: 0.2571 - accuracy: 0.9398 - val_loss: 0.1695 - val_accuracy: 0.9500


In [9]:
#      CLONE MNIST MODEL EAGER DISABLED
###########################################
importlib.reload(rsc)
importlib.reload(Clone_Model)
#K.clear_session() 

#This mixed v1 v2 TF and can only cause problems.
#-> pass eager=False to model.fit() instead!!
#tf.compat.v1.disable_eager_execution()
print(tf.executing_eagerly())

clone = Clone_Model.satify_model(model, {'run_eagerly' : True})

sat_cb = Clone_Model.sat_results()

history = clone.fit(train[0],
                    train[1],
                    batch_size=3,
                    epochs=1,
                    steps_per_epoch=3,
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[sat_cb])
                

True
<class 'tensorflow.python.keras.layers.core.Flatten'>
Clone_Model.satify_model() end.

Logs from batch callback: {'loss': 4.517240047454834, 'accuracy': 0.8333333134651184, 'o_s_sl_metric': -1.1663865540721567}

K.GETVALUE o_s satlayer: 3.703924379946267
1/3 [=========>....................] - ETA: 0s - loss: 4.5172 - accuracy: 0.8333 - o_s_sl_metric: -1.1664
Logs from batch callback: {'loss': 4.055177688598633, 'accuracy': 0.8888888955116272, 'o_s_sl_metric': 0.33361344592784326}

K.GETVALUE o_s satlayer: 6.703924379946267

Logs from batch callback: {'loss': 3.615018606185913, 'accuracy': 0.9166666865348816, 'o_s_sl_metric': 1.8336134459278435}

K.GETVALUE o_s satlayer: 9.703924379946267

Logs from epoch callback: {'loss': 3.615018606185913, 'accuracy': 0.9166666865348816, 'o_s_sl_metric': 1.8336134459278435, 'val_loss': 6.349829196929932, 'val_accuracy': 1.0, 'val_o_s_sl_metric': 7.833613445927844}

K.print_tensor()
 [0.0732487366637199 0.021791875520908777 0.048295801551710177 .

In [125]:
print(history.history.keys())
print(history.__dict__)

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
{'validation_data': None, 'model': <tensorflow.python.keras.engine.sequential.Sequential object at 0x7fe02832fc90>, '_chief_worker_only': None, 'history': {'loss': [0.19146684805552164], 'accuracy': [0.9508833], 'val_loss': [0.24173198640346527], 'val_accuracy': [0.947]}, 'params': {'batch_size': 3, 'epochs': 1, 'steps': 3, 'samples': 3, 'verbose': 1, 'do_validation': True, 'metrics': ['loss', 'accuracy', 'o_sdense', 'o_sdense_1', 'val_loss', 'val_accuracy', 'val_o_sdense', 'val_o_sdense_1']}, 'epoch': [0]}


In [ ]:
""" Warum macht tf probleme und zeigt auf auskommentierten code?
        #Default to k=1 if first EVal explains delta of variance alone 
        #TODO change condition so graph mode works without tf.function
        #if not 
        #    #cond = tf.constant([True], dtype=tf.bool)
            #print("cond with default catch: {}".format(cond))   
        """    
        
    
    #k = max(len(tf.where(cond)), 1 ) #TODO use min(1,tf.where()) instead of checking above
        
        #print("tf.where(): {}".format(tf.where(cond)))
        #print("K: in ll_fun {}".format(k))
       